# Extracting the data out of the zip

In [ ]:
# import zipfile
# import json
# import sqlite3
# import pandas as pd

# conn = sqlite3.connect("scopus.db")
# cur = conn.cursor()

# cur.execute("""
# CREATE TABLE IF NOT EXISTS papers_raw (
#     file_id INTEGER,
#     year INTEGER,
#     raw_json TEXT
# )
# """)

# conn.commit()
# conn.close()

# keep = "ScopusData2018-2023/2018/201800000"
# zip_path = "ScopusData2018-2023.zip"
# years = ["2018", "2019", "2020", "2021", "2022", "2023"]

# def loader(year: int, start_id: int, end_id: int):
#     conn = sqlite3.connect("scopus.db")
#     cur = conn.cursor()

#     with zipfile.ZipFile(zip_path, "r") as z:
#         for file_id in range(start_id, end_id + 1):
#             inner_path = f"ScopusData2018-2023/{year}/{file_id}"
#             try:
#                 with z.open(inner_path) as f:
#                     try:
#                         obj = json.load(f)
#                     except Exception:
#                         continue
#             except KeyError:
#                 continue
#             raw_text = json.dumps(obj, ensure_ascii=False)
#             cur.execute(
#                 """
#                 INSERT INTO papers_raw (file_id, year, raw_json)
#                 VALUES (?, ?, ?)
#                 """,
#                 (file_id, year, raw_text)
#             )
#     conn.commit()
#     conn.close()




In [ ]:
# from zipfile import Path

# sql_data_path = Path("scopus.db")
# # 2018
# loader(2018, 201800000, 201802761)

# # 2019
# loader(2019, 201900000, 201903081)

# # 2020
# loader(2020, 202000000, 202003392)

# # 2021
# loader(2021, 202100000, 202103814)

# # 2022
# loader(2022, 202200000, 202204243)

# # 2023
# loader(2023, 202300000, 202302889)

In [ ]:
conn = sqlite3.connect("scopus.db")

df_2018 = pd.read_sql_query(
    """
    SELECT file_id, year, raw_json
    FROM papers_raw
    WHERE year = 2018
    """,
    conn
)

conn.close()

print(df_2018.shape)
print(df_2018.head())

(2762, 3)
     file_id  year                                           raw_json
0  201800000  2018  {"abstracts-retrieval-response": {"item": {"ai...
1  201800001  2018  {"abstracts-retrieval-response": {"item": {"ai...
2  201800002  2018  {"abstracts-retrieval-response": {"item": {"ai...
3  201800003  2018  {"abstracts-retrieval-response": {"item": {"ai...
4  201800004  2018  {"abstracts-retrieval-response": {"item": {"ai...


In [2]:
import sqlite3, pandas as pd
conn = sqlite3.connect("scopus.db")

SQL = """
WITH base AS (
  SELECT
    file_id,
    year,

    -- basics
    json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-title"') AS citation_title,
    json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.abstracts')        AS abstracts,
    json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publisher.publishername') AS publishername,
    json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.sourcetitle')             AS sourcetitle,

    /* publication_date: DD/MM/YYYY */
    CASE
      WHEN json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.day')   IS NOT NULL
       AND json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.month') IS NOT NULL
       AND json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.year')  IS NOT NULL
      THEN printf('%02d/%02d/%04d',
                  CAST(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.day')   AS INTEGER),
                  CAST(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.month') AS INTEGER),
                  CAST(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.year')  AS INTEGER))
      ELSE NULL
    END AS publication_date,

    /* ce:doi -> document_classification_codes */
    COALESCE(
      json_extract(raw_json,'$."abstracts-retrieval-response".item."item-info"."itemidlist"."ce:doi"'),
      (SELECT t.value
       FROM json_tree(raw_json, '$."abstracts-retrieval-response"') AS t
       WHERE t.key = 'ce:doi'
       LIMIT 1)
    ) AS document_classification_codes,

    -- counts
    json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.tail.bibliography."@refcount"') AS refcount,
    CAST(
      COALESCE(
        json_extract(raw_json,'$."abstracts-retrieval-response".coredata."citedby-count"'),
        json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."citedby-count"'),
        (SELECT t.value
         FROM json_tree(raw_json, '$."abstracts-retrieval-response"') AS t
         WHERE t.key = 'citedby-count'
         LIMIT 1)
      ) AS INTEGER
    ) AS citedbycount,

    /* allauthors_name = JSON array of "<given> <surname>" (no degrees) */
    (
      SELECT json_group_array(name_str)
      FROM (
        SELECT
          TRIM(
            TRIM(COALESCE(json_extract(a.value,'$."preferred-name"."ce:given-name"'),
                          json_extract(a.value,'$."ce:given-name"'), ''))
            || ' ' ||
            TRIM(COALESCE(json_extract(a.value,'$."preferred-name"."ce:surname"'),
                          json_extract(a.value,'$."ce:surname"'), ''))
          ) AS name_str
        FROM json_each(
          CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".authors.author'))
            WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".authors.author')
            WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".authors.author'))
            ELSE json_array()
          END
        ) AS a
        WHERE TRIM(
                COALESCE(json_extract(a.value,'$."preferred-name"."ce:given-name"'),
                         json_extract(a.value,'$."ce:given-name"'), '') || ' ' ||
                COALESCE(json_extract(a.value,'$."preferred-name"."ce:surname"'),
                         json_extract(a.value,'$."ce:surname"'), '')
            ) <> ''
      )
    ) AS allauthors_name,

    /* categories (subject → abbrev) */
    (
      SELECT json_group_array(json_object(subject, abbrev))
      FROM (
        SELECT DISTINCT
          json_extract(sa.value,'$."$"')        AS subject,
          json_extract(sa.value,'$."@abbrev"')  AS abbrev
        FROM (
          -- primary
          SELECT * FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response"."subject-areas"."subject-area"'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response"."subject-areas"."subject-area"')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response"."subject-areas"."subject-area"'))
              ELSE json_array()
            END
          )
          UNION ALL
          -- fallback A
          SELECT * FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."subject-areas"."subject-area"'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."subject-areas"."subject-area"')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."subject-areas"."subject-area"'))
              ELSE json_array()
            END
          )
          UNION ALL
          -- fallback B
          SELECT * FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."subject-areas"."subject-area"'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".coredata."subject-areas"."subject-area"')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."subject-areas"."subject-area"'))
              ELSE json_array()
            END
          )
        ) AS sa
        WHERE subject IS NOT NULL AND abbrev IS NOT NULL
      )
    ) AS categories,

    /* creator name */
    (
      SELECT name_full
      FROM (
        SELECT TRIM(
                 COALESCE(json_extract(a.value,'$."preferred-name"."ce:given-name"'),
                          json_extract(a.value,'$."ce:given-name"'), '') || ' ' ||
                 COALESCE(json_extract(a.value,'$."preferred-name"."ce:surname"'),
                          json_extract(a.value,'$."ce:surname"'), '')
               ) AS name_full
        FROM (
          SELECT * FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'))
              ELSE json_array()
            END
          )
          UNION ALL
          SELECT * FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author'))
              ELSE json_array()
            END
          )
        ) AS a
        WHERE name_full <> ''
        LIMIT 1
      )
    ) AS creator,

    /* creator_degree */
    COALESCE(
      (
        SELECT deg FROM (
          SELECT NULLIF(TRIM(json_extract(a.value,'$."ce:degrees"')), '') AS deg
          FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'))
              ELSE json_array()
            END
          ) AS a
          UNION ALL
          SELECT NULLIF(TRIM(json_extract(a2.value,'$."ce:degrees"')), '') AS deg
          FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author'))
              ELSE json_array()
            END
          ) AS a2
        )
        WHERE deg IS NOT NULL AND deg <> ''
        LIMIT 1
      ),
      (
        SELECT NULLIF(TRIM(json_extract(ag_author.value,'$."ce:degrees"')), '')
        FROM json_each(
          CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."author-group"'))
            WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."author-group"')
            WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."author-group"'))
            ELSE json_array()
          END
        ) AS ag
        JOIN json_each(
          CASE json_type(json_extract(ag.value,'$.author'))
            WHEN 'array'  THEN json_extract(ag.value,'$.author')
            WHEN 'object' THEN json_array(json_extract(ag.value,'$.author'))
            ELSE json_array()
          END
        ) AS ag_author
        WHERE NULLIF(TRIM(json_extract(ag_author.value,'$."ce:degrees"')), '') IS NOT NULL
        AND json_extract(ag_author.value,'$."@auid"') = (
          SELECT auid FROM (
            SELECT json_extract(a.value,'$."@auid"') AS auid
            FROM json_each(
              CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'))
                WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author')
                WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'))
                ELSE json_array()
              END
            ) AS a
            UNION ALL
            SELECT json_extract(a2.value,'$."@auid"') AS auid
            FROM json_each(
              CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author'))
                WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author')
                WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author'))
                ELSE json_array()
              END
            ) AS a2
          )
          WHERE auid IS NOT NULL
          LIMIT 1
        )
        LIMIT 1
      )
    ) AS creator_degree,

    /* keywords */
    (
      SELECT json_group_array(kw_src.kw)
      FROM (
        SELECT json_extract(k.value,'$."$"') AS kw
        FROM json_each(
          CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-info"."author-keywords"."author-keyword"'))
            WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-info"."author-keywords"."author-keyword"')
            WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-info"."author-keywords"."author-keyword"'))
            ELSE json_array()
          END
        ) AS k
        UNION ALL
        SELECT json_extract(k2.value,'$."$"') AS kw
        FROM json_each(
          CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response"."authkeywords"."author-keyword"'))
            WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response"."authkeywords"."author-keyword"')
            WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response"."authkeywords"."author-keyword"'))
            ELSE json_array()
          END
        ) AS k2
      ) AS kw_src
      WHERE kw_src.kw IS NOT NULL
    ) AS keywords
  FROM papers_raw
)
SELECT
  base.*,
  COALESCE(json_array_length(base.allauthors_name), 0) AS allauthors_count
FROM base
ORDER BY year, file_id;
"""

df = pd.read_sql_query(SQL, conn)
df


,file_id,year,citation_title,abstracts,publishername,sourcetitle,publication_date,document_classification_codes,refcount,citedbycount,allauthors_name,categories,creator,creator_degree,keywords,allauthors_count
0,201800000,2018,Public health and international epidemiology f...,None,Springer International Publishing,"Radiology in Global Health: Strategies, Implem...",31/12/2018,10.1007/978-3-319-98485-8_15,76,1.0,"[""Krit Pongpirul"",""Matthew P. Lungren""]","[{""Medicine (all)"":""MEDI""}]",Krit Pongpirul,PhD..,[],2
1,201800001,2018,Flexible Printed Active Antenna for Digital Te...,"© 2018 The Institute of Electronics, Informati...",Institute of Electrical and Electronics Engine...,Progress in Electromagnetics Research Symposium,31/12/2018,10.23919/PIERS.2018.8597669,4,1.0,"[""Teerapong Pratumsiri"",""Panuwat Janpugdee""]","[{""Electrical and Electronic Engineering"":""ENG...",Teerapong Pratumsiri,None,[],2
2,201800002,2018,Parametric study of hydrogen production via so...,© 2018 Elsevier LtdComputational fluid dynamic...,Elsevier Ltd,Chemical Engineering Science,31/12/2018,10.1016/j.ces.2018.08.042,42,21.0,"[""Kiattikhoon Phuakpunk"",""Benjapon Chalermsins...","[{""Chemistry (all)"":""CHEM""},{""Chemical Enginee...",Kiattikhoon Phuakpunk,None,"[""Circulating fluidized bed"",""Computational fl...",4
3,201800003,2018,Superhydrophobic coating from fluoroalkylsilan...,© 2018 Elsevier B.V. A superhydrophobic/supero...,Elsevier B.V.,Applied Surface Science,31/12/2018,10.1016/j.apsusc.2018.08.059,45,37.0,"[""Jittraporn Saengkaew"",""Duy Le"",""Chanatip Sam...","[{""Chemistry (all)"":""CHEM""},{""Condensed Matter...",Jittraporn Saengkaew,None,"[""Encapsulation"",""Fluoroalkylsilane"",""Natural ...",8
4,201800004,2018,Electrochemical impedance-based DNA sensor usi...,© 2018 Elsevier B.V. A label-free electrochemi...,Elsevier B.V.,Analytica Chimica Acta,31/12/2018,10.1016/j.aca.2018.07.045,55,68.0,"[""Prinjaporn Teengam"",""Weena Siangproh"",""Adiso...","[{""Analytical Chemistry"":""CHEM""},{""Biochemistr...",Prinjaporn Teengam,None,"[""acpcPNA"",""Electrochemical impedance spectros...",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20181,202302885,2023,Long-chain bio-olefins production via oxidativ...,© 2021 Elsevier B.V.Long-chain α-olefins (≥ C1...,Elsevier B.V.,Catalysis Today,01/01/2023,10.1016/j.cattod.2021.07.034,63,3.0,"[""Duy Le"",""Nattaporn Chaidherasuwet"",""Atitarn ...","[{""Catalysis"":""CENG""},{""Chemistry (all)"":""CHEM""}]",Duy Le,None,"[""Long-chain olefins"",""Mesoporous KIT-6"",""Olei...",5
20182,202302886,2023,Recent Developments and Applications of Microf...,"© 2021 Taylor & Francis Group, LLC.Nowadays, f...",Taylor and Francis Ltd.,Critical Reviews in Analytical Chemistry,None,10.1080/10408347.2021.1949695,115,11.0,"[""Waleed Alahmad"",""Puttaruksa Varanusupakul"",""...","[{""Analytical Chemistry"":""CHEM""}]",Waleed Alahmad,None,"[""Biological hazards"",""chemical hazards"",""food...",3
20183,202302887,2023,"Social justice, education and peacebuilding: c...",© 2021 The Author(s). Published by Informa UK ...,Routledge,Compare,None,10.1080/03057925.2021.1951666,76,5.0,"[""Tejendra Pherali""]","[{""Education"":""SOCI""}]",Tejendra Pherali,None,"[""conflict"",""Education"",""peacebuilding"",""socia...",1
20184,202302888,2023,Effects of black soldier fly (Hermetia illucen...,© 2021 Taylor & Francis.The effects of replaci...,Taylor and Francis Ltd.,Journal of Applied Aquaculture,None,10.1080/10454438.2021.1923609,44,6.0,"[""Ratchaneegorn Mapanao"",""Wirat Jiwyam"",""Nudth...","[{""Ecology"":""ENVI""},{""Aquatic Science"":""AGRI""}]",Ratchaneegorn Mapanao,None,"[""Anabas testudineus"",""Black soldier fly"",""fis...",4


In [18]:
df_clean = df.drop(columns=["file_id","citation_title","abstracts","document_classification_codes"])

In [19]:
#Check for na values and number of rows and columns remaining
print(df_clean.shape)
df_clean.isna().sum()

(20186, 14)


year                       0
publishername             11
sourcetitle                0
publication_date        4105
refcount                   0
citedbycount               0
allauthors_name            0
categories                 0
creator                    0
creator_degree         18790
keywords                   0
allauthors_count           0
authors_cluster_raw        0
authors_level              0
dtype: int64

In [20]:
#Cleaning with publicationdate
if "publication_date" in df_clean.columns:
    df_clean["publication_date"] = pd.to_datetime(df_clean["publication_date"],format="%d/%m/%Y")
    latest_date = max(df_clean["publication_date"])
    df_clean["days_from_latest_date"] = latest_date-df_clean["publication_date"]
    df_clean.drop(columns=["publication_date"],inplace=True)
    df_clean["days_from_latest_date"].value_counts()
else:
    print("Already done")
df_clean["days_from_latest_date"].value_counts()

days_from_latest_date
395 days     463
760 days     436
729 days     309
1125 days    284
364 days     269
            ... 
249 days       1
1909 days      1
1897 days      1
1898 days      1
281 days       1
Name: count, Length: 1487, dtype: int64

In [21]:
#Cleaning for refcount. Refcount is dtype object so must be converted to int
df_clean.info()
df_clean["refcount"] = pd.to_numeric(df_clean["refcount"],errors="coerce").astype("Int64")
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20186 entries, 0 to 20185
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   year                   20186 non-null  int64          
 1   publishername          20175 non-null  object         
 2   sourcetitle            20186 non-null  object         
 3   refcount               20186 non-null  float64        
 4   citedbycount           20186 non-null  float64        
 5   allauthors_name        20186 non-null  object         
 6   categories             20186 non-null  object         
 7   creator                20186 non-null  object         
 8   creator_degree         1396 non-null   object         
 9   keywords               20186 non-null  object         
 10  allauthors_count       20186 non-null  int64          
 11  authors_cluster_raw    20186 non-null  int32          
 12  authors_level          20186 non-null  object 

In [22]:
#Cleaning for 'citedbycount'
float_exist=False
for v in df_clean["citedbycount"]:
    if pd.isna(v)==True:
        continue
    if v%1==0:
        continue
    else:
        float_exist=True
print(float_exist)
#Float value doesnt exist so we turn column into Int64
df_clean["citedbycount"] = df_clean["citedbycount"].astype("Int64")
df_clean.info()

False
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20186 entries, 0 to 20185
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   year                   20186 non-null  int64          
 1   publishername          20175 non-null  object         
 2   sourcetitle            20186 non-null  object         
 3   refcount               20186 non-null  Int64          
 4   citedbycount           20186 non-null  Int64          
 5   allauthors_name        20186 non-null  object         
 6   categories             20186 non-null  object         
 7   creator                20186 non-null  object         
 8   creator_degree         1396 non-null   object         
 9   keywords               20186 non-null  object         
 10  allauthors_count       20186 non-null  int64          
 11  authors_cluster_raw    20186 non-null  int32          
 12  authors_level          20186 non-null  o

In [23]:
import numpy as np
from sklearn.cluster import KMeans
# df = pd.read_sql_query(SQL, conn)

# --- 1) KMeans on one column (number of authors) ---
X_auth = df_clean[["allauthors_count"]].values    # shape (n_samples, 1)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_clean["authors_cluster_raw"] = kmeans.fit_predict(X_auth)   # values like 0,1,2

# --- 2) Sort clusters by center so we can say low/medium/high ---
centers = kmeans.cluster_centers_.flatten()             # length 3
order   = np.argsort(centers)                           # index of small → large

cluster_id_to_label = {
    order[0]: "low",
    order[1]: "medium",
    order[2]: "high"
}

df_clean["authors_level"] = df_clean["authors_cluster_raw"].map(cluster_id_to_label)

print(df_clean["allauthors_count"].max())
print(df_clean[["allauthors_count", "authors_cluster_raw", "authors_level"]])
print(df_clean["authors_level"].value_counts())

5246
       allauthors_count  authors_cluster_raw authors_level
0                     2                    0           low
1                     2                    0           low
2                     4                    0           low
3                     8                    0           low
4                     6                    0           low
...                 ...                  ...           ...
20181                 5                    0           low
20182                 3                    0           low
20183                 1                    0           low
20184                 4                    0           low
20185                 1                    0           low

[20186 rows x 3 columns]
authors_level
low       19666
medium      517
high          3
Name: count, dtype: int64


In [24]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

feature_cols = ["year", "citedbycount", "refcount"]   # keep it simple first

# 1) Convert to numeric (anything weird becomes NaN)
for col in feature_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

# 2) Fill missing values (needed because XGBoost hates NaN in some setups)
df_clean[feature_cols] = df_clean[feature_cols].fillna(df_clean[feature_cols].median())
y = df_clean["authors_cluster_raw"]  
# 3) Final X matrix as float
X = df_clean[feature_cols].astype(float)


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

model = XGBClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multi:softprob",
    eval_metric="mlogloss"
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3934
           1       0.44      0.07      0.12       103
           2       0.00      0.00      0.00         1

    accuracy                           0.97      4038
   macro avg       0.47      0.36      0.37      4038
weighted avg       0.96      0.97      0.96      4038



c:\Users\witta\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\witta\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\witta\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave